In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pump-fun-graduation-february-2025/chunk_40.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_23.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_18.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_21.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_38.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_37.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_36.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_14.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_6.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_35.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_34.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_32.csv
/kaggle/input/pump-fun-graduation-february-2025/token_info_onchain_divers.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_17.csv
/kaggle/input/pump-fun-graduation-february-2025/test_unlabeled.csv
/kaggle/input/pump-fun-graduation-february-2025/chunk_8.csv
/ka

In [3]:
# Cell 1: Setup and Imports
import pandas as pd
import numpy as np
import os # Useful for navigating files, especially in Kaggle environments

# Set options for better display
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [4]:
# Cell 2: Load Data
BASE_DIR = '/kaggle/input/solana-skill-sprint-memcoin-graduation/'

train_df = pd.read_csv(f'{BASE_DIR}train.csv')
test_df = pd.read_csv(f'{BASE_DIR}test_unlabeled.csv')
sample_submission_df = pd.read_csv(f'{BASE_DIR}sample_submission.csv')

print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)
print("Sample submission shape:", sample_submission_df.shape)

Train data shape: (639557, 6)
Test data shape: (478832, 4)
Sample submission shape: (478832, 2)


In [5]:
# Cell 3: Initial Data Inspection

print("\n--- Train Data Info ---")
train_df.info()

print("\n--- Train Data Head ---")
print(train_df.head())

print("\n--- Train Data Description ---")
print(train_df.describe())

print("\n--- Test Data Info ---")
test_df.info()

print("\n--- Test Data Head ---")
print(test_df.head())

print("\n--- Test Data Description ---")
print(test_df.describe())


--- Train Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639557 entries, 0 to 639556
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      639557 non-null  int64  
 1   mint            639557 non-null  object 
 2   slot_min        639557 non-null  int64  
 3   slot_graduated  7404 non-null    float64
 4   has_graduated   639557 non-null  bool   
 5   is_valid        639557 non-null  bool   
dtypes: bool(2), float64(1), int64(2), object(1)
memory usage: 20.7+ MB

--- Train Data Head ---
   Unnamed: 0                                          mint   slot_min  \
0           1  BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump  317876496   
1           3  4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump  317876500   
2           4  DeXchZLMzFm9nfvkfTKiXvBWWEpGMF6VDzvbMB6mpump  317876500   
3           5  6Mx4fnEPWpRLKtxY8VzVXo3CWwu2dvGp9BSJMWfVpump  317876500   
4           6  4rbGH5peYV3FvCf

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [6]:
# Cell 4: Data Cleaning and Filtering

# Drop the redundant 'Unnamed: 0' column
train_df = train_df.drop('Unnamed: 0', axis=1)
test_df = test_df.drop('Unnamed: 0', axis=1)

# Filter data where is_valid is True
# It's crucial to understand what 'is_valid' means. Assuming it means valid data for the competition.
train_df_valid = train_df[train_df['is_valid']].drop('is_valid', axis=1).copy()
test_df_valid = test_df[test_df['is_valid']].drop('is_valid', axis=1).copy() # Keep 'mint' for submission

print(f"Original train shape: {train_df.shape}")
print(f"Filtered train shape: {train_df_valid.shape}")
print(f"Original test shape: {test_df.shape}")
print(f"Filtered test shape: {test_df_valid.shape}")

Original train shape: (639557, 5)
Filtered train shape: (639557, 4)
Original test shape: (478832, 3)
Filtered test shape: (422180, 2)


In [7]:
# Cell 5: Target Variable Analysis

target_distribution = train_df_valid['has_graduated'].value_counts(normalize=True)

print("\n--- Target Variable Distribution (has_graduated) ---")
print(target_distribution)

# Optional: Visualize the distribution
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize=(6, 4))
# sns.countplot(x='has_graduated', data=train_df_valid)
# plt.title('Distribution of Target Variable (has_graduated)')
# plt.show()

# Check the number of positive cases
num_graduated = train_df_valid['has_graduated'].sum()
print(f"\nNumber of graduated tokens: {num_graduated}")
print(f"Percentage of graduated tokens: {target_distribution.get(True, 0) * 100:.4f}%")


--- Target Variable Distribution (has_graduated) ---
has_graduated
False    0.988423
True     0.011577
Name: proportion, dtype: float64

Number of graduated tokens: 7404
Percentage of graduated tokens: 1.1577%


In [12]:
# Cell 6: Load and Inspect Auxiliary Data (Corrected)

# Define the base directory for the auxiliary dataset
AUX_DATA_DIR = '/kaggle/input/pump-fun-graduation-february-2025/'

# List all files in the auxiliary data directory
aux_file_list = os.listdir(AUX_DATA_DIR)
print("Files in auxiliary data directory:", aux_file_list)

# Identify transaction data chunks
chunk_files = sorted([f for f in aux_file_list if f.startswith('chunk') and f.endswith('.csv')])
print(f"\nFound {len(chunk_files)} transaction data chunks.")

# Load the first chunk to inspect its structure
if chunk_files:
    first_chunk_path = f'{AUX_DATA_DIR}{chunk_files[0]}'
    print(f"\nLoading and inspecting the first chunk file: {first_chunk_path}")
    # Read only a few rows and specific columns for quick inspection
    try:
        chunk_sample_df = pd.read_csv(first_chunk_path, nrows=100) # Read first 100 rows
        print("\n--- First Chunk Data Info (Sample) ---")
        chunk_sample_df.info()
        print("\n--- First Chunk Data Head ---")
        print(chunk_sample_df.head())
    except Exception as e:
        print(f"Error reading chunk file {first_chunk_path}: {e}")

else:
    print("\nNo chunk files found in the auxiliary directory.")


# Load metadata files
dune_info_path = f'{AUX_DATA_DIR}dune_token_info.csv'
token_info_divers_path = f'{AUX_DATA_DIR}token_info_onchain_divers.csv'

print(f"\nLoading and inspecting Dune token info: {dune_info_path}")
try:
    dune_info_df = pd.read_csv(dune_info_path)
    print("\n--- Dune Token Info Info ---")
    dune_info_df.info()
    print("\n--- Dune Token Info Head ---")
    print(dune_info_df.head())
except Exception as e:
     print(f"Error reading Dune info file: {e}")


print(f"\nLoading and inspecting Onchain token info: {token_info_divers_path}")
try:
    token_info_divers_df = pd.read_csv(token_info_divers_path)
    print("\n--- Onchain Token Info Info ---")
    token_info_divers_df.info()
    print("\n--- Onchain Token Info Head ---")
    token_info_divers_df.head() # Use head() as info() was printed just above
except Exception as e:
    print(f"Error reading Onchain info file: {e}")

Files in auxiliary data directory: ['chunk_40.csv', 'chunk_23.csv', 'chunk_18.csv', 'chunk_21.csv', 'chunk_38.csv', 'chunk_37.csv', 'chunk_36.csv', 'chunk_14.csv', 'chunk_6.csv', 'chunk_35.csv', 'chunk_34.csv', 'chunk_32.csv', 'token_info_onchain_divers.csv', 'chunk_17.csv', 'test_unlabeled.csv', 'chunk_8.csv', 'chunk_7.csv', 'chunk_24.csv', 'chunk_41.csv', 'chunk_9.csv', 'chunk_33.csv', 'chunk_26.csv', 'chunk_11.csv', 'chunk_39.csv', 'chunk_10.csv', 'chunk_31.csv', 'chunk_16.csv', 'dune_token_info_v2.csv', 'chunk_4.csv', 'chunk_28.csv', 'chunk_25.csv', 'chunk_22.csv', 'chunk_2.csv', 'chunk_30.csv', 'chunk_19.csv', 'chunk_12.csv', 'dune_token_info.csv', 'train.csv', 'chunk_29.csv', 'chunk_15.csv', 'token_info_onchain_divers_v2.csv', 'chunk_1.csv', 'chunk_20.csv', 'chunk_13.csv', 'chunk_27.csv', 'chunk_5.csv', 'chunk_3.csv']

Found 41 transaction data chunks.

Loading and inspecting the first chunk file: /kaggle/input/pump-fun-graduation-february-2025/chunk_1.csv

--- First Chunk Data I

/tmp/ipykernel_31/2405911188.py:49: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  token_info_divers_df = pd.read_csv(token_info_divers_path)


In [13]:
# Cell 7: Feature Engineering Strategy - Transaction Data

# This cell outlines the strategy, the implementation will follow in later cells.

print("--- Feature Engineering Strategy from Transaction Data ---")

# The transaction data (chunk*.csv) contains records for many tokens.
# We need to join/merge this data with our train_df_valid and test_df_valid
# using the 'mint' column and potentially 'slot_min' to filter by time.

# For each unique 'mint' in our train and test sets, we will process the transaction data
# within its first 100 blocks (relative to 'slot_min').

# Potential features to extract per 'mint' from transaction data:
# 1. Transaction Counts:
#    - Total number of transactions (buys + sells)
#    - Number of buy transactions
#    - Number of sell transactions
# 2. Volume:
#    - Total SOL volume (in/out)
#    - Total token volume (in/out)
#    - Net SOL flow (SOL bought - SOL sold)
#    - Net Token flow (Tokens bought - Tokens sold)
# 3. Participants:
#    - Number of unique wallets participating (buyers + sellers)
#    - Number of unique buyers
#    - Number of unique sellers
# 4. Price/Volatility:
#    - Price at different points (e.g., start, end of 100 blocks, max, min)
#    - Price change over the 100 blocks
#    - Volatility (e.g., standard deviation of price within the window)
# 5. Balances:
#    - Final virtual SOL balance (might indicate remaining liquidity)
#    - Final virtual token balance

# Challenge: Processing potentially large chunk files efficiently.
# Strategy:
# A. Read chunks one by one.
# B. For each chunk, filter transactions that belong to the 'mint' addresses
#    in our train/test sets AND occurred within the first 100 blocks
#    after that mint's 'slot_min'.
# C. Aggregate features for each 'mint' from the filtered transactions.
# D. Concatenate aggregated features for all chunks.
# E. Merge the resulting feature dataframe with train_df_valid and test_df_valid.

print("\nNext steps will involve implementing this strategy: iterating through chunks, filtering, aggregating, and merging.")

--- Feature Engineering Strategy from Transaction Data ---

Next steps will involve implementing this strategy: iterating through chunks, filtering, aggregating, and merging.


In [20]:
# Cell 8: Implement Feature Extraction for a Single Chunk (Corrected Filtering Logic)

def extract_features_from_chunk(chunk_path, tokens_info_df):
    """
    Reads a transaction chunk file, filters relevant transactions for the tokens
    in tokens_info_df within their first 100 slots, and extracts features.
    Uses corrected column names and filtering logic.

    Args:
        chunk_path (str): The full path to the chunk CSV file.
        tokens_info_df (pd.DataFrame): DataFrame containing 'mint' and 'slot_min'
                                      for the tokens we care about (from train/test).

    Returns:
        pd.DataFrame: A DataFrame with 'mint' as index and aggregated features.
                      Returns an empty DataFrame if no relevant transactions found.
    """
    # print(f"Processing chunk: {os.path.basename(chunk_path)}") # Keep this for debugging progress

    # Read the chunk file - reading in chunks might be necessary for very large files
    try:
        # Specify relevant dtypes to potentially reduce memory usage and avoid warnings
        dtype_mapping = {
            'slot': 'int64',
            'tx_idx': 'int64', # Based on previous info, might need handling if mixed types, sticking to int for now
            'signing_wallet': 'object',
            'direction': 'object',
            'base_coin': 'object', # This is the mint address
            'base_coin_amount': 'int64', # Token amount
            'quote_coin_amount': 'int64', # SOL amount
            'virtual_token_balance_after': 'int64',
            'virtual_sol_balance_after': 'int64',
            'provided_gas_fee': 'int64',
            'fee': 'int64',
            'consumed_gas': 'int64'
            # block_time and signature can remain objects if not used for calculations
        }
        # Only load columns needed for merging and feature extraction
        cols_to_load = ['slot', 'tx_idx', 'signing_wallet', 'direction', 'base_coin',
                        'base_coin_amount', 'quote_coin_amount',
                        'virtual_token_balance_after', 'virtual_sol_balance_after'] # Added tx_idx for sorting
        chunk_df = pd.read_csv(chunk_path, dtype=dtype_mapping, usecols=cols_to_load)
    except Exception as e:
        print(f"Error reading {chunk_path}: {e}")
        return pd.DataFrame() # Return empty if read fails

    # Rename 'base_coin' to 'mint' to match the tokens_info_df and facilitate merging
    chunk_df = chunk_df.rename(columns={'base_coin': 'mint'})

    # Merge with tokens_info_df to get slot_min for filtering
    # We only keep transactions for the mints in our train/test sets
    merged_df = pd.merge(chunk_df, tokens_info_df[['mint', 'slot_min']], on='mint', how='inner')

    if merged_df.empty:
        # print("No relevant transactions found in this chunk.")
        return pd.DataFrame()

    # Filter transactions within the first 100 slots for each mint
    # CORRECTED FILTERING LOGIC HERE
    filtered_df = merged_df[
        (merged_df['slot'] >= merged_df['slot_min']) &
        (merged_df['slot'] < merged_df['slot_min'] + 100)
    ].copy() # Use .copy() to avoid SettingWithCopyWarning


    if filtered_df.empty:
        # print("No transactions within the first 100 slots in this chunk.")
        return pd.DataFrame()

    # Sort by slot and tx_idx to ensure correct 'first'/'last' for aggregation
    filtered_df = filtered_df.sort_values(by=['slot', 'tx_idx']).reset_index(drop=True)


    # --- Feature Aggregation ---
    # Group by 'mint' and calculate features
    features_df = filtered_df.groupby('mint').agg(
        n_tx=('slot', 'count'), # Number of transactions
        n_buys=('direction', lambda x: (x == 'buy').sum()), # Number of buy transactions
        n_sells=('direction', lambda x: (x == 'sell').sum()), # Number of sell transactions
        # Corrected volume calculations based on direction and amount columns
        buy_sol_volume=('quote_coin_amount', lambda x: x[filtered_df.loc[x.index, 'direction'] == 'buy'].sum()), # SOL spent on buys
        sell_sol_volume=('quote_coin_amount', lambda x: x[filtered_df.loc[x.index, 'direction'] == 'sell'].sum()), # SOL received from sells
        buy_token_volume=('base_coin_amount', lambda x: x[filtered_df.loc[x.index, 'direction'] == 'buy'].sum()), # Tokens received from buys
        sell_token_volume=('base_coin_amount', lambda x: x[filtered_df.loc[x.index, 'direction'] == 'sell'].sum()), # Tokens spent on sells

        # Note: Net flow calculations might need review depending on desired definition
        # For simplicity, let's calculate total SOL/Token traded regardless of direction for now
        # Or sum based on buy/sell volumes calculated above
        total_sol_volume = ('quote_coin_amount', 'sum'), # Sum of quote_coin_amount (SOL)
        total_token_volume = ('base_coin_amount', 'sum'), # Sum of base_coin_amount (Tokens)

        unique_wallets=('signing_wallet', 'nunique'), # Number of unique wallets in this chunk/window

        # Price approximations: SOL per Token
        # Use total volumes for a more stable price estimate within the window
        total_buy_sol_agg=('quote_coin_amount', lambda x: x[filtered_df.loc[x.index, 'direction'] == 'buy'].sum()),
        total_buy_token_agg=('base_coin_amount', lambda x: x[filtered_df.loc[x.index, 'direction'] == 'buy'].sum()),

        # Virtual balance at the end of the window (or last recorded transaction)
        # 'last' aggregation works correctly after sorting by slot/tx_idx
        last_virtual_sol_balance=('virtual_sol_balance_after', 'last'),
        last_virtual_token_balance=('virtual_token_balance_after', 'last'),

        # First/Last slot in the window for this mint within this chunk
        first_tx_slot=('slot', 'min'),
        last_tx_slot=('slot', 'max')

    ).reset_index() # Get 'mint' back as a column

    # Recalculate average price using aggregated total volumes
    features_df['avg_price_buy'] = features_df.apply(
        lambda row: row['total_buy_sol_agg'] / row['total_buy_token_agg'] if row['total_buy_token_agg'] != 0 else 0, axis=1
    )
    features_df = features_df.drop(['total_buy_sol_agg', 'total_buy_token_agg'], axis=1) # Drop intermediate columns

    # Add prefix to features to indicate source
    features_df = features_df.rename(columns={col: f'tx_{col}' for col in features_df.columns if col != 'mint'})


    return features_df

# Prepare the tokens_info_df by combining train and test mints and slot_mins
# Use the filtered dataframes from Cell 4
tokens_info_for_feature_extraction = pd.concat([
    train_df_valid[['mint', 'slot_min']],
    test_df_valid[['mint', 'slot_min']]
]).drop_duplicates(subset=['mint']).reset_index(drop=True)

print(f"\nPrepared token info for feature extraction ({len(tokens_info_for_feature_extraction)} unique mints).")


Prepared token info for feature extraction (1061737 unique mints).


In [21]:
# Cell 9: Process All Chunks and Aggregate Features (Execute Again)

# List to hold feature dataframes from each chunk
all_features_list = []

# Get the list of chunk files again (assuming Cell 6 was run)
aux_file_list = os.listdir(AUX_DATA_DIR)
chunk_files = sorted([f for f in aux_file_list if f.startswith('chunk') and f.endswith('.csv')])

print(f"Starting feature extraction from {len(chunk_files)} chunks...")

# Iterate through each chunk file
for i, chunk_file in enumerate(chunk_files):
    chunk_path = f'{AUX_DATA_DIR}{chunk_file}'
    # Call the function to extract features
    chunk_features_df = extract_features_from_chunk(chunk_path, tokens_info_for_feature_extraction)
    if not chunk_features_df.empty:
        all_features_list.append(chunk_features_df)
    # Optional: Print progress
    if (i + 1) % 5 == 0 or (i + 1) == len(chunk_files):
        print(f"Finished processing {i + 1}/{len(chunk_files)} chunks.")
    # Optional: Add a small sleep to avoid overwhelming the system or for monitoring
    # import time
    # time.sleep(1)


print("\nFinished processing all chunks. Concatenating results...")

# Concatenate all feature dataframes
if all_features_list:
    transaction_features_df = pd.concat(all_features_list, ignore_index=True)
    print("Concatenation complete.")
    print("Transaction features shape (before cross-chunk aggregation):", transaction_features_df.shape)

    # Group by mint again in case a mint appeared in multiple chunks (likely)
    # Aggregate features across chunks for the same mint
    print("Aggregating features across chunks for mints...")
    # Define aggregation methods for each column
    agg_dict = {
        col: 'sum' for col in transaction_features_df.columns if col.startswith('tx_') and col not in ['tx_last_virtual_sol_balance', 'tx_last_virtual_token_balance', 'tx_first_tx_slot', 'tx_last_tx_slot', 'tx_avg_price_buy']
    }
    # For price, avg of averages is not ideal, but sum of SOL / sum of Tokens is better
    # Since we already calculated avg_price_buy per chunk, we can take the mean, or recalculate
    # based on total buy sol/token across chunks if needed.
    # Let's stick to mean for now, or perhaps re-evaluate price calculation later.
    agg_dict['tx_avg_price_buy'] = 'mean' # Or potentially re-calculate after summing volumes
    agg_dict['tx_last_virtual_sol_balance'] = 'last' # Last recorded balance across all relevant transactions for the mint
    agg_dict['tx_last_virtual_token_balance'] = 'last' # Last recorded balance across all relevant transactions for the mint
    agg_dict['tx_first_tx_slot'] = 'min' # Earliest tx slot across all chunks for the mint
    agg_dict['tx_last_tx_slot'] = 'max' # Latest tx slot across all chunks for the mint

    transaction_features_df = transaction_features_df.groupby('mint').agg(agg_dict).reset_index()
    print("Aggregation across chunks complete.")
    print("Transaction features shape after cross-chunk aggregation:", transaction_features_df.shape)


    print("\n--- Transaction Features Head ---")
    print(transaction_features_df.head())
    print("\n--- Transaction Features Info ---")
    transaction_features_df.info()
    # print("\n--- Transaction Features Description ---") # Might be too verbose
    # print(transaction_features_df.describe())


else:
    transaction_features_df = pd.DataFrame()
    print("No transaction features extracted.")

Starting feature extraction from 41 chunks...
Finished processing 5/41 chunks.
Finished processing 10/41 chunks.
Finished processing 15/41 chunks.
Finished processing 20/41 chunks.
Finished processing 25/41 chunks.
Finished processing 30/41 chunks.
Finished processing 35/41 chunks.
Finished processing 40/41 chunks.
Finished processing 41/41 chunks.

Finished processing all chunks. Concatenating results...
Concatenation complete.
Transaction features shape (before cross-chunk aggregation): (1062257, 16)
Aggregating features across chunks for mints...
Aggregation across chunks complete.
Transaction features shape after cross-chunk aggregation: (1061737, 16)

--- Transaction Features Head ---
                                          mint  tx_n_tx  tx_n_buys  \
0  112GXdaoTJN4nrL2P2Gc3vXKSkHmTfzvLMBGJukpump       61         36   
1  114Y1wSoqDdPZpcb8kDedZ1KU1bvmVXAJdNidGRpump       10          6   
2  115Xxvg1WzerrEwjgDaVL93uULKko8Z1QLDMD5Epump       90         54   
3  115vjkpCJMXwikx4tr

In [23]:
# Cell 10: Merge Features and Prepare Data for Modeling (Corrected Column Names for NaN Fill)

# Merge transaction features into train and test dataframes
print("Merging transaction features with train and test data...")

# Left merge to keep all original train/test tokens, even those with no transactions
train_merged_df = pd.merge(train_df_valid, transaction_features_df, on='mint', how='left')
test_merged_df = pd.merge(test_df_valid, transaction_features_df, on='mint', how='left')

print(f"Train merged shape: {train_merged_df.shape}")
print(f"Test merged shape: {test_merged_df.shape}")

# Identify columns with NaN values introduced by the merge (mints with no transactions)
# Make sure transaction_features_df is not empty before getting columns
if not transaction_features_df.empty:
    new_feature_columns = transaction_features_df.columns.difference(['mint'])
    print(f"\nNew feature columns added: {list(new_feature_columns)}")

    # Check for NaNs in the new feature columns
    print("\nNaN counts in new features (Train):")
    print(train_merged_df[new_feature_columns].isnull().sum().sort_values(ascending=False).head())

    print("\nNaN counts in new features (Test):")
    print(test_merged_df[new_feature_columns].isnull().sum().sort_values(ascending=False).head())

    # Handle NaNs - A common approach is imputation (e.g., filling with 0 for count/volume features)
    # For count/volume features, NaN likely means 0 activity in the first 100 blocks.
    # For balance features, NaN might need a different imputation or handling.
    # Let's fill most new features with 0.
    # CORRECTED: Use the prefixed column names for balance features
    balance_cols = ['tx_last_virtual_sol_balance', 'tx_last_virtual_token_balance']
    features_to_fill_zero = new_feature_columns.difference(balance_cols)

    train_merged_df[features_to_fill_zero] = train_merged_df[features_to_fill_zero].fillna(0)
    test_merged_df[features_to_fill_zero] = test_merged_df[features_to_fill_zero].fillna(0)

    # CORRECTED: Use the prefixed column names for balance features when filling NaNs
    train_merged_df[balance_cols] = train_merged_df[balance_cols].fillna(0)
    test_merged_df[balance_cols] = test_merged_df[balance_cols].fillna(0)

else:
     print("\nNo transaction features dataframe to merge.")
     # If transaction_features_df is empty, train_merged_df and test_merged_df are just train_df_valid and test_df_valid
     train_merged_df = train_df_valid.copy()
     test_merged_df = test_df_valid.copy()


# Define features (X) and target (y)
TARGET = 'has_graduated'
# Drop mint, slot_min, slot_graduated (if present in train), and any other non-feature columns
features = [col for col in train_merged_df.columns if col not in [TARGET, 'mint', 'slot_min', 'slot_graduated']]

X_train = train_merged_df[features]
y_train = train_merged_df[TARGET]
X_test = test_merged_df[features] # We need mint for submission later, but not for prediction. Keep test_merged_df for submission.

print(f"\nFeatures for training ({len(features)}):", features)
print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")

# Display head of prepared data
print("\n--- X_train Head ---")
print(X_train.head())

print("\n--- X_test Head ---")
print(X_test.head())

# Store the test_merged_df for later submission
test_merged_df_for_submission = test_merged_df.copy()

# Note: Metadata features from dune_token_info and token_info_onchain_divers
# are not included yet. That would be another feature engineering step if desired.

Merging transaction features with train and test data...
Train merged shape: (639557, 19)
Test merged shape: (422180, 17)

New feature columns added: ['tx_avg_price_buy', 'tx_buy_sol_volume', 'tx_buy_token_volume', 'tx_first_tx_slot', 'tx_last_tx_slot', 'tx_last_virtual_sol_balance', 'tx_last_virtual_token_balance', 'tx_n_buys', 'tx_n_sells', 'tx_n_tx', 'tx_sell_sol_volume', 'tx_sell_token_volume', 'tx_total_sol_volume', 'tx_total_token_volume', 'tx_unique_wallets']

NaN counts in new features (Train):
tx_avg_price_buy       0
tx_buy_sol_volume      0
tx_buy_token_volume    0
tx_first_tx_slot       0
tx_last_tx_slot        0
dtype: int64

NaN counts in new features (Test):
tx_avg_price_buy       0
tx_buy_sol_volume      0
tx_buy_token_volume    0
tx_first_tx_slot       0
tx_last_tx_slot        0
dtype: int64

Features for training (15): ['tx_n_tx', 'tx_n_buys', 'tx_n_sells', 'tx_buy_sol_volume', 'tx_sell_sol_volume', 'tx_buy_token_volume', 'tx_sell_token_volume', 'tx_total_sol_volume',

In [25]:
# Cell 11: Model Selection

import lightgbm as lgb
# import xgboost as xgb # XGBoost is another strong alternative

print("Selecting LightGBM Classifier as the model.")
# We will instantiate the model in the next cell where we train it,
# potentially setting parameters like objective, metrics, and class_weight.

Selecting LightGBM Classifier as the model.


In [26]:
# Cell 12: Model Training

# Calculate scale_pos_weight for handling imbalance
neg_count = y_train.value_counts().get(False, 0)
pos_count = y_train.value_counts().get(True, 0)
scale_pos_weight_value = neg_count / pos_count if pos_count > 0 else 1

print(f"Calculated scale_pos_weight: {scale_pos_weight_value}")

# Initialize the LightGBM Classifier model
# Using parameters often found effective in similar competitions.
# 'objective': 'binary' for binary classification
# 'metric': 'logloss' as per the competition evaluation metric
# 'n_estimators': Number of boosting rounds (can be tuned)
# 'learning_rate': Step size shrinkage (can be tuned)
# 'num_leaves': Complexity of the tree (can be tuned)
# 'seed': for reproducibility
# 'n_jobs': for parallel processing
# 'scale_pos_weight': to handle class imbalance

lgb_clf = lgb.LGBMClassifier(objective='binary',
                             metric='logloss',
                             n_estimators=1000, # A starting point, consider early stopping
                             learning_rate=0.05,
                             num_leaves=31, # Default, can be tuned
                             max_depth=-1, # No limit
                             random_state=42,
                             n_jobs=-1,
                             scale_pos_weight=scale_pos_weight_value, # Address imbalance
                             colsample_bytree=0.8, # Feature fraction
                             subsample=0.8, # Data fraction
                             reg_alpha=0.1, # L1 regularization
                             reg_lambda=0.1 # L2 regularization
                            )

print("Starting model training...")

# Train the model
lgb_clf.fit(X_train, y_train)

print("Model training complete.")

# Note: For better evaluation and to prevent overfitting,
# cross-validation and early stopping should be used in practice.
# This is a basic training step for now.

Calculated scale_pos_weight: 85.37992976769314
Starting model training...
[LightGBM] [Info] Number of positive: 7404, number of negative: 632153
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.094233 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3745
[LightGBM] [Info] Number of data points in the train set: 639557, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011577 -> initscore=-4.447111
[LightGBM] [Info] Start training from score -4.447111
Model training complete.


In [27]:
# Cell 13: Make Predictions

print("Making predictions on the test set...")

# Predict probabilities. The output is an array of shape (n_samples, 2),
# where the second column ([[:, 1]]) is the probability of the positive class (True, i.e., graduated).
test_probabilities = lgb_clf.predict_proba(X_test)[:, 1]

print("Predictions complete.")
print("Shape of test probabilities:", test_probabilities.shape)
print("Sample test probabilities:", test_probabilities[:10])

Making predictions on the test set...
Predictions complete.
Shape of test probabilities: (422180,)
Sample test probabilities: [0.30599323 0.02892616 0.11633548 0.25011308 0.16772698 0.25895146
 0.08822064 0.10529137 0.03042206 0.32733643]


In [28]:
# Cell 14: Generate Submission File

print("Generating submission file...")

# Create a submission DataFrame
# The sample submission has columns 'mint' and 'has_graduated'.
submission_df = pd.DataFrame({'mint': test_merged_df_for_submission['mint'],
                              'has_graduated': test_probabilities})

# Save the submission file
submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Submission file created successfully: {submission_path}")
print("\n--- Submission File Head ---")
print(submission_df.head())

Generating submission file...
Submission file created successfully: submission.csv

--- Submission File Head ---
                                           mint  has_graduated
0  9Wt3N7etKMX9cioTdEJ5S4b8A9nK3M66n9RFVgBGpump       0.305993
1  9q5y2X2P8ZEKTjyXBVcS5q2EZM7HbNV8DURY2qnvqi2f       0.028926
2  HL2di8dcQ7eYDmkcFoZ4zJyHX5SbRZXAJxTegL3JPfx2       0.116335
3  7iAFj9Pc5QH9jbGmHwYe8T6yzNVbjhL13PNJXVTspump       0.250113
4  F7U1Rdgz2KFpneKpAnYytWF2jggnsrLScfi2A668pump       0.167727
